# ICN Programming Course — Short Introduction to Git (for Jupyter users)
*Prepared on 2025-09-22*

## What you'll learn
- What Git is and why to use it
- Core concepts: repo, commits, staging area, branches; local vs remote
- Essential commands you can run directly from a Jupyter notebook
- Working with GitHub: remotes, cloning, pushing, pulling
- Handling merge conflicts (guided demo)
- Good practices: README, LICENSE, commit messages
- Mini-exercises you can do on your own

> **Tip:** In Jupyter, prefix shell commands with `!` or use a `%%bash` cell magic.


## 1. What is Git (in one paragraph)?
Git is a *distributed* version-control system that records snapshots of your project over time. 
It works best with plain-text files (e.g., `.py`, `.md`, `.txt`) and enables you to explore history, branch safely, and collaborate without overwriting each other.
**Git ≠ GitHub**: GitHub is a hosting platform that stores remote repositories and adds collaboration tools (issues, pull requests, CI, security checks).


## 2. Install and configure (once per machine)
If you're on a managed lab machine, Git may already be installed. Otherwise:
- **Windows**: download from https://git-scm.com/download/win (select *Use Git from Windows Command Prompt*).
- **macOS**: type `git` in Terminal and follow the prompts to install Xcode command line tools.
- **Linux**: use your package manager (`apt`, `dnf`, etc.).

After installing, set your identity and a default editor (done once):


## Configuring Git (first-time setup)

Before you start using Git, you should tell it *who you are* and set a default editor.  
This information is stored once on your computer (global configuration), and will be used in every repository you create.

- **`user.name`**: Your full name, used to label each commit you make.  
- **`user.email`**: Your email address, also attached to commits (it should match the one you use with GitHub if you want commits linked to your profile).  
- **`core.editor`**: The text editor Git opens when it needs you to type a commit message or resolve a merge conflict. You can set this to something simple like `nano` (terminal editor), or to your preferred editor (`code` for VS Code, `subl` for Sublime Text, etc.).  

> 📝 **Note:** You only need to do this setup **once per computer**. After that, Git will remember your settings for all repositories on that machine.


In [ ]:




# Configure with your details (run once)
!git config --global user.name "Your Name"
!git config --global user.email "your.name@your.domain"
# Choose an editor you are comfortable with (nano, code, subl, etc.)
!git config --global core.editor "nano"
# (Optional) show the current config
!git config --list


diff.astextplain.textconv=astextplain
filter.lfs.clean=git-lfs clean -- %f
filter.lfs.smudge=git-lfs smudge -- %f
filter.lfs.process=git-lfs filter-process
filter.lfs.required=true
http.sslbackend=schannel
core.autocrlf=true
core.fscache=true
core.symlinks=false
core.editor="C:\\Program Files\\Notepad++\\notepad++.exe" -multiInst -notabbar -nosession -noPlugin
pull.rebase=false
credential.helper=manager
credential.https://dev.azure.com.usehttppath=true
init.defaultbranch=main
user.name=Your Name
user.email=your.name@your.domain
filter.lfs.smudge=git-lfs smudge -- %f
filter.lfs.process=git-lfs filter-process
filter.lfs.required=true
filter.lfs.clean=git-lfs clean -- %f
core.editor=nano


## 3. Create a local repository
We'll make a sample folder and initialise Git.


In [ ]:

%%bash
set -e
# Create and enter a practice folder (safe to re-run)
mkdir -p ~/icn_git_intro && cd ~/icn_git_intro
# Initialise a repo (creates a hidden .git directory)
git init
# Show hidden items; you should now see .git/
ls -a


### 3.1 Add a first file, stage, and commit
Git tracks *staged* changes. Use `git add` to stage, then `git commit` to record a snapshot.


In [ ]:

%%bash
set -e
cd ~/icn_git_intro
# Create a README in Markdown
echo "# My First Git Repo" > README.md
echo "" >> README.md
echo "This is a demo repository created from a Jupyter notebook." >> README.md

# See status
git status

# Stage and commit
git add README.md
git commit -m "Add initial README with project description"

# View history
git log --oneline


### 3.2 Inspect differences
`git diff` shows *unstaged* changes; `git diff --cached` shows staged changes. You can also compare commits.


In [ ]:

%%bash
set -e
cd ~/icn_git_intro
# Make an edit
echo "Second line: more details." >> README.md

# View unstaged diff
git diff | sed -n '1,120p'

# Stage and view staged diff
git add README.md
git diff --cached | sed -n '1,120p'

# Commit the change
git commit -m "Append details to README"
git log --oneline


## 4. Branches (safe parallel work)
Create a branch to try changes without touching `main`. Merge it back later.


In [ ]:

%%bash
set -e
cd ~/icn_git_intro

# Create and switch to a new branch
git checkout -b feature/title-tweak

# Change the title
sed -i.bak '1s/.*/# My First Git Repository (ICN demo)/' README.md || true
# (macOS sed fallback)
[ -f README.md.bak ] && mv README.md.bak README.md || true

git add README.md
git commit -m "Tweak README title on feature branch"

# Switch back and merge
git checkout main
git merge feature/title-tweak --no-edit

# Show a compact history graph
git log --oneline --graph --decorate


## 5. Connect to GitHub (remote)
1. Create an **empty** repo on GitHub (do **not** initialise with README/LICENCE).
2. Copy its HTTPS URL, e.g. `https://github.com/USERNAME/icn_git_intro.git`.
3. Add it as a remote and push your local `main` branch.
> Authentication: GitHub requires a **Personal Access Token (PAT)** instead of your password for HTTPS pushes. Store it via the credential helper so you are not prompted every time.


In [ ]:

%%bash
set -e
cd ~/icn_git_intro

# Replace the URL with your own remote repo URL:
REMOTE_URL="https://github.com/USERNAME/icn_git_intro.git"

# Add and verify the remote (safe to re-run)
git remote remove origin 2>/dev/null || true
git remote add origin "$REMOTE_URL"
git remote -v

# (Optional) cache credentials so you don't have to retype the PAT:
git config --global credential.helper store

# First push; sets upstream
git push -u origin main || echo "Note: This will prompt for GitHub username and PAT if run interactively."


### 5.1 Pull, fetch, and push
- `git fetch` downloads changes from the remote but doesn't modify your files.
- `git pull` = fetch + merge (integrates remote changes).
- `git push` uploads your local commits to the remote.


In [ ]:

%%bash
set -e
cd ~/icn_git_intro

# Fetch (view remote branches/updates without merging)
git fetch

# Pull (merge remote changes into your current branch)
git pull --ff-only || echo "If you have divergent history, you may need to resolve or rebase."

# Push local commits to the remote
git push


## 6. Practice merge conflicts (guided)
We'll create a small conflict by editing the same line on two branches in different ways.


In [ ]:

%%bash
set -e
cd ~/icn_git_intro

# Ensure we are on main
git checkout main

# Create an alternate branch and change a line to 'SLC'
git checkout -b new_feature
sed -i.bak '1s/.*/# Hello, SLC!/' README.md || true
[ -f README.md.bak ] && mv README.md.bak README.md || true
git commit -am "Change greeting to SLC on new_feature"

# Switch back and change the same line differently
git checkout main
sed -i.bak '1s/.*/# Hello, Salt Lake City!/' README.md || true
[ -f README.md.bak ] && mv README.md.bak README.md || true
git commit -am "Change greeting to Salt Lake City on main"

# Attempt to merge -> expect a conflict
set +e
git merge new_feature
RET=$?
set -e

if [ $RET -ne 0 ]; then
  echo ">>> Merge conflict created intentionally. Open README.md and resolve the conflict markers."
fi

# Show what conflict markers look like:
echo '
<<<<<<< HEAD
# Hello, Salt Lake City!
=======
# Hello, SLC!
>>>>>>> new_feature
' | sed -n '1,120p'


### 6.1 Resolve and conclude the merge
Open `README.md`, choose the final text (e.g., `# Hello, Salt Lake City!`), *remove* the conflict markers, then:


In [ ]:

%%bash
set -e
cd ~/icn_git_intro

# Resolve programmatically for the demo (keep the long form)
echo "# Hello, Salt Lake City!" > README.md

git add README.md
git commit -m "Resolve merge conflict in README (choose full city name)"

# Push the resolved merge
git push


## 7. Undo & restore (use carefully)
- Discard local file changes: `git restore <file>`
- Unstage a staged file: `git restore --staged <file>`
- Revert a commit by *adding* an inverse commit: `git revert <COMMIT>`
- Forcefully align your local branch with remote (DANGER: destructive): `git reset --hard origin/main`


In [ ]:

%%bash
set -e
cd ~/icn_git_intro

# Example: make a quick change, then restore
echo "Temporary line to be discarded." >> README.md
git status -s
git restore README.md
git status -s


## 8. Good practice
- **Small, focused commits** with meaningful messages
- Always start projects with a **README.md**
- Include a **LICENSE** file so others know how they can use your work
- Keep secrets out of version control; use `.gitignore` for data dumps, caches, etc.
- Sync frequently: pull before you push


In [ ]:

%%bash
set -e
cd ~/icn_git_intro

# Add a minimal LICENSE (MIT as example) and a .gitignore
cat > LICENSE <<'MIT'
MIT License

Copyright (c) 2025

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

[...trimmed for brevity in class demo...]
MIT

cat > .gitignore <<'IGN'
# Python
__pycache__/
*.py[cod]
.venv/
.env

# Jupyter
.ipynb_checkpoints/

# Data (example)
data/
IGN

git add LICENSE .gitignore
git commit -m "Add LICENSE (MIT, demo) and .gitignore"
git push || true


## 9. Mini‑exercises (to try now or later)
1. Create a new repo locally, add a couple of files, and commit twice. Use `git log` to inspect history.
2. Create and switch to a new branch, make edits, and merge back to `main`.
3. Create an empty remote on GitHub for that project, add `origin`, and push `main`.
4. On GitHub, edit the README online, then `git fetch` and `git pull` locally to get the change.
5. Intentionally create a conflict (as above) and practice resolving it.
6. Add a `LICENSE` and `.gitignore`, commit, and push.


---
### Appendix: Useful references
- Pro Git (free book): https://git-scm.com/book
- Git docs: https://git-scm.com/doc
- Software Carpentry Git lesson: https://swcarpentry.github.io/git-novice/
- Markdown guide: https://www.markdownguide.org/getting-started/

> If you prefer GUIs, try **GitHub Desktop** or the **VS Code Git** panel.
